# U-Net Model Training Script

**Author:** Prisca Dotti

**Last Edit:** 23.10.2023

This Jupyter Notebook contains the code for training a U-Net model on a dataset of sparks videos. The dataset is split into training and testing sets, and the model is trained using the training set. The testing set is used to evaluate the performance of the trained model.

To run the notebook, simply execute each cell in order.

In [1]:
# autoreload is used to reload modules automatically before entering the
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# To import modules from parent directory in Jupyter Notebook
import sys

sys.path.append("..")

In [2]:
import logging
import os

import torch
from torch import nn, optim
from torch.backends import cudnn

# from torch.cuda.amp import GradScaler
from torch.utils.data import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter

import wandb
from config import TrainingConfig, config
from models.UNet import unet
from utils.training_inference_tools import (
    MyTrainingManager,
    sampler,
    test_function,
    training_step,
    weights_init,
)
from utils.training_script_utils import (
    get_sample_ids,
    init_config_file_path,
    init_criterion,
    init_dataset,
    init_model,
)

logger = logging.getLogger(__name__)

In [3]:
##################### Get training-specific parameters #####################

# Initialize training-specific parameters
# (get the configuration file path from ArgParse)
config_filename = os.path.join("config_files", "config_final_model.ini")
params = TrainingConfig(training_config_file=config_filename)

# Print parameters to console if needed
params.print_params()

######################### Initialize random seeds ##########################

# We used these random seeds to ensure reproducibility of the results

# torch.manual_seed(0) <--------------------------------------------------!
# random.seed(0) <--------------------------------------------------------!
# np.random.seed(0) <-----------------------------------------------------!

[12:31:20] [  INFO  ] [   config   ] <291 > -- Loading C:\Users\dotti\sparks_project\config_files\config_final_model.ini
[12:31:20] [  INFO  ] [   config   ] <533 > --     training_config_file: C:\Users\dotti\sparks_project\config_files\config_final_model.ini
[12:31:20] [  INFO  ] [   config   ] <533 > --              dataset_dir: C:\Users\dotti\sparks_project\data\sparks_dataset
[12:31:20] [  INFO  ] [   config   ] <533 > --                        c: <configparser.ConfigParser object at 0x0000027893A46AD0>
[12:31:20] [  INFO  ] [   config   ] <533 > --                 run_name: final_model
[12:31:20] [  INFO  ] [   config   ] <533 > --            load_run_name: 
[12:31:20] [  INFO  ] [   config   ] <533 > --               load_epoch: 0
[12:31:20] [  INFO  ] [   config   ] <533 > --             train_epochs: 90000
[12:31:20] [  INFO  ] [   config   ] <533 > --                criterion: lovasz_softmax
[12:31:20] [  INFO  ] [   config   ] <533 > --                 lr_start: 0.0001
[12:31

In [4]:
############################ Configure datasets ############################

# Select samples for training and testing based on dataset size
train_sample_ids = get_sample_ids(
    train_data=True,
    dataset_size=params.dataset_size,
)
test_sample_ids = get_sample_ids(
    train_data=False,
    dataset_size=params.dataset_size,
)

# Initialize training dataset
dataset = init_dataset(
    params=params,
    sample_ids=train_sample_ids,
    apply_data_augmentation=True,
    load_instances=False,
)

# Initialize testing datasets
testing_dataset = init_dataset(
    params=params,
    sample_ids=test_sample_ids,
    apply_data_augmentation=False,
    load_instances=True,
)

[12:31:20] [  INFO  ] [utils.training_script_utils] <137 > -- Samples in training dataset: 9
[12:31:24] [  INFO  ] [utils.training_script_utils] <137 > -- Samples in training dataset: 22


In [5]:
# train with only one batch
# ids = np.arange(0, params.batch_size, 1, dtype=np.int64)
# dataset = torch.utils.data.Subset(dataset, ids)

In [6]:
# Initialize data loaders
dataset_loader = DataLoader(
    dataset,
    batch_size=params.batch_size,
    shuffle=True,
    num_workers=params.num_workers,
    pin_memory=params.pin_memory,
)

In [7]:
############################## Configure UNet ##############################

# Initialize the UNet model
network = init_model(params=params)

# Move the model to the GPU if available
if params.device != "cpu":
    network = nn.DataParallel(network).to(params.device, non_blocking=True)
    # cudnn.benchmark = True

# Watch the model with wandb for logging if enabled
if params.wandb_log:
    wandb.watch(network)

# Initialize UNet weights if required
if params.initialize_weights:
    logger.info("Initializing UNet weights...")
    network.apply(weights_init)

# The following line is commented as it does not work on Windows
# torch.compile(network, mode="default", backend="inductor")

[12:31:24] [  INFO  ] [  __main__  ] < 17 > -- Initializing UNet weights...


In [8]:
########################### Initialize training ############################

# Initialize the optimizer based on the specified type
if params.optimizer == "adam":
    optimizer = optim.Adam(network.parameters(), lr=params.lr_start)
elif params.optimizer == "adadelta":
    optimizer = optim.Adadelta(network.parameters(), lr=params.lr_start)
else:
    logger.error(f"{params.optimizer} is not a valid optimizer.")
    exit()

# Initialize the learning rate scheduler if specified
if params.scheduler == "step":
    scheduler = optim.lr_scheduler.StepLR(
        optimizer,
        step_size=params.scheduler_step_size,
        gamma=params.scheduler_gamma,
    )
else:
    scheduler = None

# Define the output directory path
output_path = os.path.join(config.output_dir, params.run_name)
logger.info(f"Output directory: {os.path.realpath(output_path)}")

# Initialize the summary writer for TensorBoard logging
summary_writer = SummaryWriter(os.path.join(
    output_path, "summary"), purge_step=0)

# Check if a pre-trained model should be loaded
if params.load_run_name != "":
    load_path = os.path.join(config.output_dir, params.load_run_name)
    logger.info(f"Model loaded from directory: {os.path.realpath(load_path)}")
else:
    load_path = None

# Initialize the loss function
criterion = init_criterion(params=params, dataset=dataset)

# Create a directory to save predicted class movies
preds_output_dir = os.path.join(output_path, "predictions")
os.makedirs(preds_output_dir, exist_ok=True)

# Create a dictionary of managed objects
managed_objects = {"network": network, "optimizer": optimizer}
if scheduler is not None:
    managed_objects["scheduler"] = scheduler

# Create a training manager with the specified training and testing functions
trainer = MyTrainingManager(
    # Training parameters
    training_step=lambda _: training_step(
        dataset_loader=dataset_loader,
        params=params,
        sampler=sampler,
        network=network,
        optimizer=optimizer,
        criterion=criterion,
        scheduler=scheduler,
        # scaler=GradScaler(),
    ),
    save_every=params.c.getint("training", "save_every", fallback=5000),
    load_path=load_path,
    save_path=output_path,
    managed_objects=unet.managed_objects(managed_objects),
    # Testing parameters
    test_function=lambda _: test_function(
        network=network,
        device=params.device,
        criterion=criterion,
        params=params,
        testing_dataset=testing_dataset,
        training_name=params.run_name,
        output_dir=preds_output_dir,
        training_mode=True,
        debug=config.debug_mode,
    ),
    test_every=params.c.getint("training", "test_every", fallback=1000),
    plot_every=params.c.getint("training", "test_every", fallback=1000),
    summary_writer=summary_writer,
)

# Load the model if a specific epoch is provided
if params.load_epoch != 0:
    trainer.load(params.load_epoch)

[12:31:24] [  INFO  ] [  __main__  ] < 24 > -- Output directory: C:\Users\dotti\sparks_project\models\saved_models\final_model


In [9]:
# torch.set_float32_matmul_precision("high")

In [10]:
############################## Start training ##############################

# Set the network in training mode
network.train()


# Resume the W&B run if needed (commented out for now)
# if wandb.run.resumed:
#     checkpoint = torch.load(wandb.restore(checkpoint_path))
#     network.load_state_dict(checkpoint['model_state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#     epoch = checkpoint['epoch']
#     loss = checkpoint['loss']

# Check if training is enabled in the configuration
if params.c.getboolean("general", "training", fallback=False):
    # Validate the network before training if resuming from a checkpoint
    if params.load_epoch > 0:
        logger.info("Validate network before training")
        trainer.run_validation(wandb_log=params.wandb_log)

In [11]:
# Check if training is enabled in the configuration
if params.c.getboolean("general", "training", fallback=False):
    logger.info("Starting training")
    # Train the model for the specified number of epochs
    trainer.train(
        params.train_epochs,
        print_every=params.c.getint("training", "print_every", fallback=100),
        wandb_log=params.wandb_log,
    )

# Check if final testing/validation is enabled in the configuration
if params.c.getboolean("general", "testing", fallback=False):
    logger.info("Starting final validation")
    # Run the final validation/testing procedure
    trainer.run_validation(wandb_log=params.wandb_log)

# Close the summary writer
summary_writer.close()

# Close the wandb run
if params.wandb_log:
    wandb.finish()

[12:31:25] [  INFO  ] [  __main__  ] < 3  > -- Starting training
[12:31:48] [WARNING ] [utils.training_inference_tools] <136 > -- loss: 0.7503355741500854, loss is tensor: True
[12:31:48] [WARNING ] [utils.training_inference_tools] <139 > -- gradients zeroed
[12:31:57] [WARNING ] [utils.training_inference_tools] <142 > -- loss backward
[12:31:57] [WARNING ] [utils.training_inference_tools] <145 > -- optimizer step done
[12:31:57] [WARNING ] [utils.training_inference_tools] <159 > -- End of training step
[12:32:34] [  INFO  ] [utils.training_inference_tools] <1314> -- Iteration 0...
[12:32:34] [  INFO  ] [utils.training_inference_tools] <1315> -- 	Training loss: 0.7503
[12:32:34] [  INFO  ] [utils.training_inference_tools] <1316> -- 	Time elapsed: 70.12s
[12:32:46] [WARNING ] [utils.training_inference_tools] <136 > -- loss: 0.7502722144126892, loss is tensor: True
[12:32:46] [WARNING ] [utils.training_inference_tools] <139 > -- gradients zeroed
[12:32:46] [WARNING ] [utils.training_infe

In [ ]:
# For debugging purposes
# model_parameters = filter(lambda p: p.requires_grad, network.parameters())
# model_parameters = sum([np.prod(p.size()) for p in model_parameters])
# logger.debug(f"Number of trainable parameters: {model_parameters}")

In [ ]:
# for load_epoch in [10000,20000,30000,40000,50000,60000,70000,80000,90000,100000]:
# for load_epoch in [100000]:
#     trainer.load(load_epoch)
#     logger.info("Starting final validation")
#     trainer.run_validation(wandb_log=wandb_log)
# if wandb_log:
#     wandb.finish()

### Visualize UNet architecture (for debugging)

In [ ]:
# # get number of trainable parameters
# num_params = sum(p.numel() for p in network.parameters() if p.requires_grad)
# logger.debug(f"Number of trainable parameters: {num_params}")
# # get dummy unet input
# batch = next(iter(dataset_loader))
# x = batch[0].to(device)
# yhat = network(x[:,None]) # Give dummy batch to forward()
# from torchviz import make_dot
# make_dot(yhat, params=dict(list(network.named_parameters()))).render("unet_model", format="png")
# a = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]

# len(a[0:4])